## Exercises - Constrained Optimization

### Data Processing

In [3]:
# NOTE: !!! THIS CELL SHOULD BE RUN FIRST !!!

# Jupyter notebooks have a bad interaction with virtual environments: 
# they perceive the PYTHONPATH as the folder the venv was activated
# instead of whatever the current directory of the .ipynb file is
# so we need to incorporate this hack

import sys

filepath = "/Users/colinyao/Desktop/Code/FINM/Portfolio_36700/HW/PortfolioRiskHW/HW2"
# change this path to whatever the path to this file is on your computer
sys.path.append(filepath)

import os
os.chdir(filepath)

In [4]:
import pandas as pd

# Input names for files and sheets
INFILE = 'data/spx_returns_weekly.xlsx'
SHEET_INFO = 's&p500 names'
SHEET_RETURNS = 's&p500 rets'
SHEET_BENCH = 'benchmark rets'

# Weekly frequency
FREQ = 52

# Tickers of interest
TICKS =  ['AAPL','NVDA','MSFT','GOOGL','AMZN','META','TSLA','AVGO','BRK/B','LLY']
TICK_ETF = 'SPY'

In [6]:
info = pd.read_excel(INFILE,sheet_name=SHEET_INFO)
info.set_index('ticker',inplace=True)
temp_mkt_cap = info.loc[TICKS].copy()
temp_mkt_cap['mkt cap'] /= 1e9
temp_mkt_cap.rename(columns={'mkt cap':'mkt cap (billions $)'},inplace=True)
temp_mkt_cap.style.format({'mkt cap (billions $)':'{:,.0f}'})

,name,mkt cap (billions $)
ticker,,
AAPL,Apple Inc,"3,009"
NVDA,NVIDIA Corp,"3,480"
MSFT,Microsoft Corp,"3,514"
GOOGL,Alphabet Inc,"2,146"
AMZN,Amazon.com Inc,"2,304"
META,Meta Platforms Inc,"1,745"
TSLA,Tesla Inc,994
AVGO,Broadcom Inc,"1,149"
BRK/B,Berkshire Hathaway Inc,"1,064"


In [7]:
# Get returns data for the tickers of interest
rets = pd.read_excel(INFILE,sheet_name=SHEET_RETURNS)
rets.set_index('date',inplace=True)
rets = rets[TICKS]

In [8]:
# Get benchmark returns (SPY)
bench = pd.read_excel(INFILE,sheet_name=SHEET_BENCH)
bench.set_index('date',inplace=True)
rets[TICK_ETF] = bench[TICK_ETF]

### Risk Return Metrics

In [9]:
# Define a function to compute performance metrics
import numpy as np

def performanceMetrics(returns, annualization=1, quantile=.05):
    """Compute performance metrics for a set of asset returns
    """
    metrics = pd.DataFrame(index=returns.columns)
    metrics['Mean'] = returns.mean() * annualization
    metrics['Vol'] = returns.std() * np.sqrt(annualization)
    metrics['Sharpe'] = (returns.mean() / returns.std()) * np.sqrt(annualization)
    metrics['Min'] = returns.min()
    metrics['Max'] = returns.max()
    return metrics

In [10]:
# Get performance metrics for the assets
mets = performanceMetrics(rets, annualization=FREQ)
mets.style.format('{:.1%}')

,Mean,Vol,Sharpe,Min,Max
AAPL,23.9%,27.7%,86.3%,-17.5%,14.7%
NVDA,64.6%,46.3%,139.3%,-20.1%,30.2%
MSFT,26.1%,24.0%,108.9%,-14.4%,15.0%
GOOGL,21.7%,28.0%,77.5%,-12.0%,25.8%
AMZN,29.3%,30.6%,95.9%,-14.5%,18.5%
META,26.2%,35.1%,74.6%,-23.7%,24.5%
TSLA,47.0%,58.6%,80.1%,-25.9%,33.3%
AVGO,39.5%,37.5%,105.3%,-18.3%,25.2%
BRK/B,13.5%,19.1%,70.8%,-13.4%,9.8%
LLY,28.2%,28.3%,99.5%,-12.2%,17.5%


### Constrained Optimization

Continue working with the data above. Suppose we want to constrain the weights such that

- there are no short positions beyond negative `20%`, for all
- none of the positions may have weight over `35%`, for all .
- all the asset weights must sum to 1

Furthermore,

- The targeted mean return is `20%` per year.
- Be careful; the target is an annualized mean.

In [11]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Prepare the inputs for optimization

FREQ = 52 # weekly to annual
mean = rets.mean() * FREQ
cov = rets.cov() * FREQ
TARGET_MEAN = 0.20  # 20% per year
N = len(mean)

# Initial guess (equal weights)
w0 = np.ones(N) / N

# Define the objective function
def objective(w):
    return w.T @ cov @ w

# Equality constraints
def fun_constraint_capital(w):
    return np.sum(w) - 1

def fun_constraint_mean(w):
    return (mean @ w) - TARGET_MEAN

constraint_capital = {'type': 'eq', 'fun': fun_constraint_capital}
constraint_mean = {'type': 'eq', 'fun': fun_constraint_mean}

constraints = ([constraint_capital, constraint_mean])

# Bounds for inequality constraints
bounds = [(-0.20, 0.35) for _ in range(N)]

#### **1.1.**

Report the weights of the constrained portfolio.

Report the mean, volatility, and Sharpe ratio of the resulting portfolio.

In [12]:
opt = minimize(
    objective,
    w0,
    bounds=bounds,
    constraints=constraints,
    method='SLSQP',
    tol=1e-12
)

if opt.success:
    print('Optimization SUCCESSFUL.')
else:
    print('Optimization FAILED.')

print(f'Iterations: {opt.nit}.')

w_constrained = opt.x

port_mean = mean @ w_constrained
port_vol = np.sqrt(w_constrained.T @ cov @ w_constrained)
port_sharpe = port_mean / port_vol

results_constrained = pd.Series({
    'Mean': port_mean,
    'Volatility': port_vol,
    'Sharpe': port_sharpe
})

pd.DataFrame({'Bounded Weights': w_constrained}, index=mean.index) \
    .sort_values('Bounded Weights', ascending=False) \
    .style.format("{:.2%}")

Optimization SUCCESSFUL.
Iterations: 35.


,Bounded Weights
BRK/B,35.00%
LLY,21.33%
SPY,15.08%
MSFT,14.52%
AMZN,9.34%
AVGO,3.62%
AAPL,2.96%
GOOGL,0.89%
META,0.24%
NVDA,-1.36%


#### **1.2.**

Compare these weights to the assets’ Sharpe ratios and means.

Do the most extreme positions also have the most extreme Sharpe ratios and means?

Why?

In [13]:
pd.DataFrame({
    'Mean': mets['Mean'],
    'Sharpe': mets['Sharpe'],
    'Bounded Weights': w_constrained
    }) \
    .sort_values('Sharpe', ascending=False) \
    .style.format("{:.2%}")

,Mean,Sharpe,Bounded Weights
NVDA,64.56%,139.35%,-1.36%
MSFT,26.14%,108.93%,14.52%
AVGO,39.49%,105.26%,3.62%
LLY,28.15%,99.49%,21.33%
AMZN,29.34%,95.90%,9.34%
AAPL,23.87%,86.29%,2.96%
TSLA,46.98%,80.10%,-1.62%
GOOGL,21.68%,77.47%,0.89%
SPY,13.13%,76.82%,15.08%
META,26.19%,74.55%,0.24%


The most extreme position does NOT necessarily have the most extreme Sharpe ratios and means. A high Sharpe ratio asset can still get a small (or even negative) weight if it is highly correlated with other assets, i.e. its marginal contribution to diversification is low. This is because the mean-variance optimizer values diversification, not just individual performance.

#### **1.3**

Compare the bounded portfolio weights to the unbounded portfolio weights (obtained from optimizing without the inequality constraints, keeping the equality constraints.)

Report the mean, volatility, and Sharpe ratio of both

In [14]:
# Unbounded Portfolio
opt_unbounded = minimize(
    objective,
    w0,
    constraints=constraints,
    method='SLSQP',
    tol=1e-12
)

if opt_unbounded.success:
    print('Optimization SUCCESSFUL.')
else:
    print('Optimization FAILED.')

print(f'Iterations: {opt_unbounded.nit}.')

w_unbounded = opt_unbounded.x

port_mean_unbounded = mean @ w_unbounded
port_vol_unbounded = np.sqrt(w_unbounded.T @ cov @ w_unbounded)
port_sharpe_unbounded = port_mean_unbounded / port_vol_unbounded

results_unbounded = pd.Series({
    'Mean': port_mean_unbounded,
    'Volatility': port_vol_unbounded,
    'Sharpe': port_sharpe_unbounded
})

Optimization SUCCESSFUL.
Iterations: 40.


In [15]:
# Compare bounded vs unbounded
compare_weights = pd.DataFrame({
    'Bounded Weighs': w_constrained,
    'Unbounded Weights': w_unbounded,
    'Sharpe Ratio': mets['Sharpe']
}, index=mean.index)

compare_weights = compare_weights.sort_values('Sharpe Ratio', ascending=False)

compare_weights.style.format({
    'Bounded Weights': '{:.2%}',
    'Unbounded Weights': '{:.2%}',
    'Sharpe Ratio': '{:.2%}'
})

,Bounded Weighs,Unbounded Weights,Sharpe Ratio
NVDA,-0.013603,-1.44%,139.35%
MSFT,0.145178,14.61%,108.93%
AVGO,0.036174,3.63%,105.26%
LLY,0.213318,21.11%,99.49%
AMZN,0.093426,9.38%,95.90%
AAPL,0.029585,2.99%,86.29%
TSLA,-0.016155,-1.54%,80.10%
GOOGL,0.008858,0.97%,77.47%
SPY,0.150846,12.69%,76.82%
META,0.002373,0.30%,74.55%


In [16]:
pd.concat([results_constrained, results_unbounded], axis=1,
           keys=['Bounded', 'Unbounded']).style.format("{:.2%}")

,Bounded,Unbounded
Mean,20.00%,20.00%
Volatility,16.49%,16.49%
Sharpe,121.29%,121.31%
